In [1]:
%reload_ext autoreload
%autoreload 2

import realsense_photo as rsp
import CustomDataset as cdset

from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device("cuda:0") # Let's make sure GPU is available!

In [2]:
pictures, labels = rsp.getDataVector("home")
dataset = cdset.MyDataset(pictures, labels, transform = transforms.ToTensor())

In [26]:
#train_count = int(len(dataset)/5*4)
#test_count = len(dataset) - train_count
#train_set, test_set = torch.utils.data.random_split(dataset, [train_count, test_count])

In [7]:
batch_size = 64

data_size = len(dataset)
validation_split = .2
split = int(np.floor(validation_split * data_size))
indices = list(range(data_size))
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         sampler=val_sampler)

In [10]:
num_epochs = 5 
num_classes = 10
learning_rate = 0.001

class ConvNet(nn.Module): 
     def __init__(self): 
        super(ConvNet, self).__init__() 
        self.layer1 = nn.Sequential( nn.Conv2d(4, 32, kernel_size=5, stride=1, padding=2), 
           nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.layer2 = nn.Sequential( nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
           nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
        self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(480*640*24, 1000) 
        self.fc2 = nn.Linear(1000, 10)
        pass
     def forward(self, x): 
        out = self.layer1(x) 
        out = self.layer2(out) 
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out) 
        out = self.fc1(out) 
        out = self.fc2(out) 
        return out
    

In [ ]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [41]:
total_step = len(train_loader)
loss_list = []
acc_list = []
model = model.float()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Прямой запуск
        outputs = model(images.float())
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6x1228800 and 3136x1000)